In [1]:
import pandas as pd 
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import root_mean_squared_error

## Q1. Downloading the data

We'll use [the same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page),
but instead of "**Green** Taxi Trip Records", we'll use "**Yellow** Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

* 16
* 17
* 18
* 19

In [2]:
train = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
train.columns.shape[0]

19

## Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes. 

What's the standard deviation of the trips duration in January?

* 32.59
* 42.59
* 52.59
* 62.59

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [4]:
# compute duration using dropoff and pickup time
# convert into minutes by getting total time in seconds and divide by 60
def get_duration(df):
    df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
    df['duration'] = df['duration'].dt.total_seconds() / 60
    return df['duration']

In [5]:
train['duration'] = get_duration(train)
train['duration'].std()

42.594351241920904

## Q3. Dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

* 90%
* 92%
* 95%
* 98%

In [6]:
total_rows_before = train.shape[0]
# double check that there are outliers by looking at min and max durations
train['duration'].describe()

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
25%      7.116667e+00
50%      1.151667e+01
75%      1.830000e+01
max      1.002918e+04
Name: duration, dtype: float64

In [7]:
def drop_outliers(df):
    return df[(df['duration'] >= 1) & (df['duration'] <= 60)]

df = drop_outliers(train)
print(f'Fraction: {(df.shape[0] * 100)/total_rows_before :.0f}%')

Fraction: 98%


## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model. 

* Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will 
  label encode them)
* Fit a dictionary vectorizer 
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

* 2
* 155
* 345
* 515
* 715

In [8]:
def ohe(df):
    # Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will 
    # label encode them)
    id_list = ['DOLocationID', 'PULocationID']
    df = df[id_list].astype(str)
    dict_df = df.to_dict(orient='records')
    
    return dict_df

In [9]:
# Fit a dictionary vectorizer
# Get a feature matrix from it
dv = DictVectorizer()
dict_df = ohe(df)
X_train = dv.fit_transform(dict_df)
y_train = df['duration'].values

# dimensionality of this feature matrix (num. of columns)
X_train.shape[1]

515

## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model. 

* Train a plain linear regression model with default parameters, where duration is the response variable
* Calculate the RMSE of the model on the training data

What's the RMSE on train?

* 3.64
* 7.64
* 11.64
* 16.64

In [10]:
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_train)
root_mean_squared_error(y_train, y_pred)

7.649261927686161

## Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023). 

What's the RMSE on validation?

* 3.81
* 7.81
* 11.81
* 16.81

In [11]:
val = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')
val['duration'] = get_duration(val)

# drop outliers
val = drop_outliers(val)
# one hot encoding 
dict_val = ohe(val)
y_test = val['duration'].values
X_test = dv.transform(dict_val)
# predict on transformed validation data 
val_pred = model.predict(X_test)
# compute RMSE 
root_mean_squared_error(y_test, val_pred)

7.811817957524739